In [88]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [89]:
%cd drive/MyDrive/to_Sedat_for_testing/Teknofest_Segmentation_PNG

[Errno 2] No such file or directory: 'drive/MyDrive/to_Sedat_for_testing/Teknofest_Segmentation_PNG'
/content/drive/MyDrive/to_Sedat_for_testing/Teknofest_Segmentation_PNG


In [90]:
!ls

Oturum2
Oturum2_Output_Ready_To_TEST
Segmentation_pre-processing.ipynb
TEKNOFEST_UnetPlusPlus_timm-efficientnet-b6_Dict.pt
test
testing-unet-senet154.ipynb
Unet_Senet154_Dict_mIoU-0.804.pt


In [91]:
import cv2
import os
import time
from PIL import Image
import numpy as np

inputdir = './Oturum2'
outdir = './Oturum2_Output_Ready_To_TEST/'

In [92]:
def focus_actual_image(image):
    
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    
    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        
        break
        
    return ROI,x,y,w,h


def png2ROI():
  i=0
  for image in os.listdir(inputdir):
      file_name   = image.split(".")[0]
      #print(file_name)
      ROI,x,y,w,h = focus_actual_image(inputdir+"/{}".format(image))
    
      try:
          mask = np.asarray(Image.open(inputdir+"/{}.png".format(file_name)))
          roi_mask= mask[y:y+h, x:x+w]
          #print(roi_mask.shape)
          roi_mask = Image.fromarray(roi_mask)
          #print(type(roi_mask))
          roi_mask.save("./{}/{}.png".format(outdir,file_name))
      except Exception as e:
          print(f"{e} bir hata oluştu")
      
      i+=1
      if i%50 ==0 :
          time.sleep(1)


In [93]:
png2ROI()

In [94]:
target_size = 512  # 512x512

def padding_zeros():

    for image in os.listdir(outdir):
        file_name   = image.split(".")[0]
        img = Image.open(outdir + file_name + ".png")
        width, height = img.size
       
        if width < 512  or height < 512:
          left_side = int((target_size - width) / 2)
          top_side = int((target_size - height) / 2)
          print(img.mode)

          result = Image.new(img.mode, (target_size, target_size), (0,0,0))
          result.paste(img, (left_side, top_side))
          result.save("{}{}.png".format(outdir,file_name))
        else:
          print("else")
          img = img.resize((512,512))
          img.save("{}{}.png".format(outdir,file_name))

In [95]:
padding_zeros()

RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
RGBA
